In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import sys
sys.path.insert(0,'../')
from lightroot.lineage import *
#data frame with t, id
# def matrix_life_mask(times,max_time):
#     ar = np.zeros(max_time)
#     ar[times] = 1
#     return ar

# def make_life_mat(df):
#     maxt = df.t.dropna().max()+1
#     index = []
#     mat = []
#     for k,g in df.groupby("key"):
#         l = list(g.t.dropna().values)
#         vals = matrix_life_mask(l, maxt)
#         index.append(k)
#         mat.append(vals)
#     return pd.DataFrame(mat, index=index)

# def make_fluc_mat(df):
#     return df[["epsilon", "key", "t"]].groupby(["t", "key"])[["epsilon"]].\
#     mean().reset_index().pivot("key", "t", "epsilon").fillna(0)

05/02/2018 09:09:37 INFO(None):loaded lightroot


In [6]:
df=pd.read_csv("../cached_data/tracked_blobs.csv", comment='#')
df

,t,key,x,y,z,epsilon
0,1,0,148.0,48.5,4.5,0.0
1,1,2,180.5,286.5,4.5,0.0
2,1,1,291.5,231.5,5.0,0.0


In [7]:

df.t =df.t+1

fluc_mat = make_fluc_mat(df)
life_mat = make_life_mat(df)

bandd = life_mat.diff(1,axis=1,).fillna(0) #* lifetime_matrix
births = bandd[bandd==1].fillna(0)
deaths = np.abs(bandd[bandd==-1]).fillna(0)
ageMat = life_mat.cumsum(axis=1) * life_mat
KEY = "310717_Run0140"
life_mat.to_csv("../cached_data/lifemat_{}.txt".format(KEY),index=None)

In [ ]:
life_mat

In [ ]:

restricted = life_mat.sum(axis=1).reset_index()#.set_index("index")
restricted = life_mat[restricted[0]>1]
restricted.head()

In [ ]:
ageMat

In [ ]:
# chk=df[20:].reset_index().drop("Unnamed: 0",1).drop("s", 1).drop("level_0",1).drop("index",1)
# chk.epsilon = chk.epsilon.fillna(0).round(4)
# chk[["key","t", "x", "y", "z", "epsilon"]].to_csv("raw.csv", index=None)

In [ ]:
life_mat.astype(int).to_csv("life_matrix.csv", index=None)

In [ ]:
fluc_mat.mean().plot(figsize=(20,10))

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(20,10))
sns.heatmap(fluc_mat,)

In [ ]:
plt.figure(figsize=(20,10))
#life_mat_snap = make_life_mat(df[(df.t> 20) &(df.t< 50)])
sns.heatmap(life_mat)

In [ ]:
plt.figure(figsize=(20,10))
#life_mat_snap = make_life_mat(df[(df.t> 20) &(df.t< 50)])
sns.heatmap(life_mat)
restricted.index.name="idx"
wind_restricted = restricted[(restricted.index > 80) &(restricted.index < 600)]
sns.heatmap(wind_restricted)


In [ ]:
time = life_mat.sum(axis=0).reset_index().drop("index",1)
time.columns = ["alive"]
time["births"] = births.sum()
time["deaths"] = deaths.sum()
time["observed_by_t"] = births.sum().cumsum().astype(int)
time["mean_age_at_t"]  = ageMat.replace(0, np.nan).mean().round(2).fillna(0)
time.index.name = 't'
time = time.reset_index()
time
#time.to_csv("timeseries.csv",index=None)

In [ ]:
#alive
life_mat.sum(axis=0).plot(title="alive",figsize=(20,10))

In [ ]:
births.sum().plot(title="births",figsize=(20,10))

In [ ]:
births.sum().cumsum().plot(title="total cells",figsize=(20,10))

In [ ]:
mean_age = ageMat.replace(0, np.nan).mean()
mean_age.plot(figsize=(20,10))

In [ ]:
restricted = life_mat.sum(axis=1).reset_index()#.set_index("index")
restricted = life_mat[restricted[0]>1]
restricted.head()

In [ ]:
bandd = wind_restricted.diff(1,axis=1,).fillna(0) #* lifetime_matrix
births = bandd[bandd==1].fillna(0)
births.sum().plot(title="births",figsize=(20,10))

In [ ]:
ts = births.sum().reset_index()
ts["minutes"]=ts["index"] * 15
ts["hours"] = ts["minutes"] / 60.
ts["days"] = ts["hours"] / 24.
ts = ts.rename(columns={0:"val"})
ts  = ts[["val", "hours", "minutes"]].set_index("hours")
ts

In [ ]:
ts["pseudoDate"] = "1 jan 2012"
ts.pseudoDate = pd.to_datetime(ts.pseudoDate)
ts["pseudoDate"] = ts.pseudoDate + pd.to_timedelta(ts.minutes, "m")
ts = ts.set_index("pseudoDate")[["val"]]
ts = ts[ts >0].dropna()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):

    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=12)
    rolstd = pd.rolling_std(timeseries, window=12)

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 4))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
test_stationarity(ts.val.values)

In [ ]:
hts = ts.resample("h").mean()
hts.plot()


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(ts.val.values,freq=4*12)
fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(15, 8)


In [ ]:
decomposition = seasonal_decompose(ts.val.values, freq=20*4)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 8)

In [ ]:
temp = pd.DataFrame(decomposition.seasonal)
temp[temp> 2.5].dropna()

In [ ]:
108/4

In [ ]:
from pandas.plotting import autocorrelation_plot
plt.figure(figsize=(20,10))
autocorrelation_plot(ts.val)

In [ ]:
test = pd.DataFrame(index=ts.index).reset_index().set_index("hours")
import numpy as np
test["random"] = np.random.normal(5, 2, len(test)) 
test[::24*4] += 50
autocorrelation_plot(test.random)

In [ ]:
dec = seasonal_decompose(test.random.values, freq=24*4)

In [ ]:
fig = plt.figure()  
fig = dec.plot()  
fig.set_size_inches(15, 8)